In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import requests


def getData(ticker='X:BTCUSD', timespan='minute', limit=50000):
    api_key = 'api-key'
    from_date = datetime.now() 
    to_date = from_date + timedelta(days=1) 
    to_date = to_date.strftime('%Y-%m-%d') 
    from_date = from_date.strftime('%Y-%m-%d')
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{from_date}/{to_date}?adjusted=true&sort=asc&limit={limit}&apiKey={api_key}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['results'])
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df.rename(columns={'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v': 'Volume'}, inplace=True)
    df.drop(columns=['vw', 'n', 't'], inplace=True)
    return df

data = getData()
data

In [2]:
def signals(df, window=20, num_std=2):
    df['SMA'] = df['Close'].rolling(window).mean()
    df['STD'] = df['Close'].rolling(window).std()

    df['Upper Band'] = df['SMA'] + (df['STD'] * num_std)
    df['Lower Band'] = df['SMA'] - (df['STD'] * num_std)

    df['Buy Signal'] = (df['Close'] < df['Lower Band'])   
    df['Sell Signal'] = (df['Close'] > df['Upper Band']) 

    return df

data_w_signals = signals(data)
data_w_signals

,Volume,Open,Close,High,Low,timestamp,SMA,STD,Upper Band,Lower Band,Buy Signal,Sell Signal
0,6.926938,96536.93,96402.74,96579.0,96402.74,2025-02-08 00:00:00,NaN,NaN,NaN,NaN,False,False
1,3.138690,96405.99,96474.38,96508.0,96397.37,2025-02-08 00:01:00,NaN,NaN,NaN,NaN,False,False
2,1.553925,96480.61,96469.93,96543.0,96446.44,2025-02-08 00:02:00,NaN,NaN,NaN,NaN,False,False
3,2.737826,96469.37,96419.80,96534.0,96403.21,2025-02-08 00:03:00,NaN,NaN,NaN,NaN,False,False
4,3.600032,96419.80,96500.00,96534.0,96419.80,2025-02-08 00:04:00,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1435,6.902669,96588.34,96481.29,96685.0,96481.28,2025-02-08 23:55:00,96638.4335,69.110178,96776.653857,96500.213143,True,False
1436,2.963106,96488.06,96503.30,96611.0,96453.86,2025-02-08 23:56:00,96632.5650,75.395398,96783.355795,96481.774205,False,False
1437,2.197687,96503.29,96486.87,96598.0,96486.85,2025-02-08 23:57:00,96623.9360,81.763423,96787.462847,96460.409153,False,False
1438,7.622836,96486.89,96462.38,96598.0,96459.84,2025-02-08 23:58:00,96611.0550,86.015157,96783.085313,96439.024687,False,False


In [3]:
to_date = datetime.now() 
from_date = to_date - timedelta(days=1) 
to_date = to_date.strftime('%Y-%m-%d') 
from_date = from_date.strftime('%Y-%m-%d')

print(to_date)
print(from_date)

2025-02-09
2025-02-08
